<a href="https://colab.research.google.com/github/rnlima8/DSWP/blob/master/Notebooks/NB23__Tools%20para%20DataViz_RNL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entendendo os dados
***

## Objetivo

* Explorar e entender os diferentes bias presentes nos dados;
* Explorar as features e identificar potenciais Bias antes de treinar o ML;
* Avaliar a performance por subgrupos antes de agregar.

## Sobre o dataframe

Neste exercício nós vamos trabalhar com o dataframe...

### Features numéricas:
Numeric Features

### Features categóricas

### Objetivo
O objetivo é estimar a probabilidade de churn.

### Label (variável-target)
*   `churn`: indicador se o cliente deixou ou não a empresa.






## Setup: Carregar as libraries necessárias para a análise
* Precisamos usar o Tensorflow 2.x.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import rcParams

In [2]:
import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.3.0'

A seguir, vamos usar uma ferramenta opensource chamada [Facets](https://pair-code.github.io/facets/), criada pela [PAIR](https://research.google/teams/brain/pair/). Facets possui 2 importantes ferramentas para entendermos os dados e, consequentemente, nos ajudar com o Machine Learning.

### Ajustar a granularidade do report

In [3]:
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

from google.colab import widgets

# Facets
from IPython.core.display import display, HTML
import base64

!pip install facets-overview==1.0.0
from facets_overview.feature_statistics_generator import FeatureStatisticsGenerator

### Carregar os dados
* Vamos carregar os dados do Kaggle/LabData.

In [26]:
url_treinamento = 'https://raw.githubusercontent.com/MathMachado/DataFrames/master/adult.data'
url_teste = 'https://raw.githubusercontent.com/MathMachado/DataFrames/master/adult.test'

In [27]:
nomes_colunas = ["Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Marital Status",
                         "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss",
                         "Hours per week", "Country", "Target"]

In [28]:
df_50T = pd.read_csv(url_treinamento, na_values="?", names = nomes_colunas, sep = ', ')
df_50V = pd.read_csv(url_teste, na_values="?", names = nomes_colunas, sep = ', ')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


Substituir o ? por np.nan

In [29]:
# df_50T.select_dtypes('float')
df_50T.select_dtypes

<bound method DataFrame.select_dtypes of        Age         Workclass  fnlwgt  ... Hours per week        Country Target
0       39         State-gov   77516  ...             40  United-States  <=50K
1       50  Self-emp-not-inc   83311  ...             13  United-States  <=50K
2       38           Private  215646  ...             40  United-States  <=50K
3       53           Private  234721  ...             40  United-States  <=50K
4       28           Private  338409  ...             40           Cuba  <=50K
...    ...               ...     ...  ...            ...            ...    ...
32556   27           Private  257302  ...             38  United-States  <=50K
32557   40           Private  154374  ...             40  United-States   >50K
32558   58           Private  151910  ...             40  United-States  <=50K
32559   22           Private  201490  ...             20  United-States  <=50K
32560   52      Self-emp-inc  287927  ...             40  United-States   >50K

[32561 row

In [30]:
colunas_numericas = list(df_50T.select_dtypes('int64').columns)
colunas_numericas

['Age',
 'fnlwgt',
 'Education-Num',
 'Capital Gain',
 'Capital Loss',
 'Hours per week']

In [31]:
colunas_categoricas = list(df_50T.select_dtypes('object').columns)
colunas_categoricas.remove('Target')
colunas_categoricas

['Workclass',
 'Education',
 'Marital Status',
 'Occupation',
 'Relationship',
 'Race',
 'Sex',
 'Country']

In [15]:
# df_50T[df_50T['Age'] == '?'] #  = np.nan

,Age,Workclass,fnlwgt,Education,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Target


In [22]:
# for coluna in colunas_numericas:
#  df_50T = np.nan

In [32]:
df_50T.head()

,Age,Workclass,fnlwgt,Education,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Target
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [33]:
df_50T.shape

(32561, 15)

## Analisando os dados com Facets
* Como dito anteriormente, é muito importante entendermos os dados (80% do tempo) antes da fase de modelagem.

Responda as perguntas a seguir:

* Existem missing values?
* Há alguma variável com valor inesperado?
* Há sinais de distorção (Skewness) nos dados?

A seguir, um pequeno overview do [Facets Overview](https://pair-code.github.io/facets/), uma ferramenta iterativa para Data Visualization para nos ajudar a explorar os dados.

In [34]:
fsg = FeatureStatisticsGenerator()
dataframes = [
    {'table': df_50T, 'name': 'trainData'}]

censusProto = fsg.ProtoFromDataFrames(dataframes)
protostr = base64.b64encode(censusProto.SerializeToString()).decode("utf-8")

HTML_TEMPLATE = """<script src="https://cdnjs.cloudflare.com/ajax/libs/webcomponentsjs/1.3.3/webcomponents-lite.js"></script>
        <link rel="import" href="https://raw.githubusercontent.com/PAIR-code/facets/1.0.0/facets-dist/facets-jupyter.html">
        <facets-overview id="elem"></facets-overview>
        <script>
          document.querySelector("#elem").protoInput = "{protostr}";
        </script>"""
html = HTML_TEMPLATE.format(protostr=protostr)
display(HTML(html))

### Ações
* O que fazer com os missing values?
    * Podemos ignorar os missing values?
* O que fazer com os outliers?
* O que fazer com as variáveis que possuem distribuições muito diferentes da Normalidade?

## A Deeper Dive

Depois das primeiras impressões acerca dos dados, é hora de darmos um mergulho mais profundo (Deeper Dive). Vamos usar a segunda ferramenta chamada [Facets Dive](https://pair-code.github.io/facets/), uma ferramenta iterativa para DataVuz que nos ajudará a entender ainda mais nossos dados.

In [37]:
#@title Número de pontos para visualização no facets

# SAMPLE_SIZE = df_50T.shape[0] #@param

SAMPLE_SIZE = 5000 #@param

train_dive = df_50T.sample(SAMPLE_SIZE).to_json(orient = 'records')

HTML_TEMPLATE = """<script src="https://cdnjs.cloudflare.com/ajax/libs/webcomponentsjs/1.3.3/webcomponents-lite.js"></script>
        <link rel="import" href="https://raw.githubusercontent.com/PAIR-code/facets/1.0.0/facets-dist/facets-jupyter.html">
        <facets-dive id="elem" height="600"></facets-dive>
        <script>
          var data = {jsonstr};
          document.querySelector("#elem").data = data;
        </script>"""
html = HTML_TEMPLATE.format(jsonstr = train_dive)
display(HTML(html))

## Exercícios

1. No menu **Binning | X-Axis** selecione **education**, e no menu **Color By**, selecione 'target' e no menu **Label By** selecione **income_bracket**.

* Como você descreveria o relacionamento entre estas variáveis?